In [1]:
import os
import torch
import torchvision
import torch.nn.functional as nnF
import torch.nn as nn
import torchtoolbox.transform as transforms
import torch.optim as optim
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import time
from torchvision import models

In [2]:
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [3]:
labels_name = ['AD', 'AV', 'LP', 'OCH', 'TCA', 'UKN']

In [4]:
checkpoint_path = "E:/Skin_App/MobileNet_v2_final/best-checkpoint/"



class MyMobilenet (nn.Module):

    def __init__(self, mobilenet, num_class, freeze_conv=False, p_dropout=0.5,
                 comb_method=None, comb_config=None, n_feat_conv=1280):

        super(MyMobilenet, self).__init__()

        _n_meta_data = 0
        if comb_method is not None:
            
            if comb_method == 'concat':
                if not isinstance(comb_config, int):
                    raise Exception("comb_config must be int for 'concat' method")
                _n_meta_data = comb_config
                self.comb = 'concat'
            else:
                raise Exception("There is no comb_method called " + comb_method + ". Please, check this out.")
        else:
            self.comb = None

        self.features = nn.Sequential(*list(mobilenet.children())[:-1])
        # freezing the convolution layers
        if freeze_conv:
            for param in self.features.parameters():
                param.requires_grad = False

        self.classifier = nn.Linear(1280 + 20, 6)

    def forward(self, img, meta_data=None):

        # Checking if when passing the metadata, the combination method is set
        if meta_data is not None and self.comb is None:
            raise Exception("There is no combination method defined but you passed the metadata to the model!")
        if meta_data is None and self.comb is not None:
            raise Exception("You must pass meta_data since you're using a combination method!")

        x = self.features(img)
        x = x.mean([2, 3])

        if self.comb == 'concat':
            x = x.view(x.size(0), -1) # flatting
            x = torch.cat([x, meta_data], dim=1) # concatenation
        return self.classifier(x)


In [7]:
train_csv_folder = "E:/Skin_App/Drive/ked_parsed_train.csv"
def get_labels_frequency (data_csv, col_target, col_single_id, verbose=False):
    # Loading the data_csv
    if isinstance(data_csv, str):
        data_csv = pd.read_csv(data_csv)

    data_ = data_csv.groupby([col_target])[col_single_id].count()
    if (verbose):
        print('### Data summary: ###')
        print(data_)
        print(">> Total samples: {} <<".format(data_.sum()))

    return data_
ser_lab_freq = get_labels_frequency(train_csv_folder, "Diagnosis", "Image")
labels_name = ser_lab_freq.index.values
freq = ser_lab_freq.values
weights = "frequency"
if weights == 'frequency':
        weights = (freq.sum() / freq).round(3)
loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor(weights))
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [8]:
device = torch.device("cpu")
model = MyMobilenet(models.mobilenet_v2(), 6)
ckpt = torch.load('E:/Skin_App/MobileNet_v2_final/best-checkpoint/best-checkpoint.pth')
model.load_state_dict(ckpt['model_state_dict'])
optimizer.load_state_dict(ckpt['optimizer_state_dict'])
epoch = ckpt['epoch']
loss_fn = ckpt['loss']
model.eval()

MyMobilenet(
  (features): Sequential(
    (0): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

In [13]:
site_dict = {'Abdomain':2, 'Anterior_Torso':3,'Armpit':4, 'Chin':5, 'Ear':6, 'Forhead':7,'Lateral_Face':8,'Lower_Back':9, 
             'Lower_Extremity':10, 'Nail':11, 'Navel':12, 'Neck':13, 'Nose':14, 'Perioribital':15, 'Posterior_Torso':16, 
             'Scalp':17, 'Upper_Extremity':18}
def predict(img,age,Gender,site):
    
    meta_data = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],dtype=np.float32)
    if Gender == 'F':
        meta_data [0] = 1.0
    else:
        meta_data [0] = 0.0
    meta_data [1] = age
    meta_data[site_dict[site]] = 1
    #img = Image.open(img)
    img = test_transform(img)
    img = torch.tensor(img, device=torch.device("cpu"), dtype=torch.float)
    meta_data = torch.tensor(meta_data, device=torch.device("cpu"), dtype=torch.float)
    img = img[None,:,:,:]
    meta_data = meta_data[None,:]
    inputs = (img,meta_data)
    #inputs = torch.tensor(inputs)
    preds = model(inputs)
    preds = torch.softmax(preds)
    return preds.detach().numpy()[0][0] * 100

In [16]:
from PIL import Image
import cv2
site = 'Nail'
Gender = 'M'
age = '28'
img = Image.open('E:/Skin_App/Drive/Images/AD0.jpg').convert('RGB')
img = np.array(img)
k = predict(img, age, Gender,site)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not tuple